<a href="https://colab.research.google.com/github/Rocks-n-Code/PythonCourse/blob/master/6%20-%20Scraping%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I covered this notebook for Denver Data Drivers and you can follow along with the video [here.](https://www.youtube.com/watch?v=cO8fWCPp_6k)

---

# Scraping Data

I imagine not all of you are working for super majors and have access to every log or dataset known to man.  That also being said I don't think your boss is going to let you buy a thousand digital logs from *a vendor at $150 a pop for a regional study to support a prospect.  Your tech, if you have access to one, is also going to want to murder you if you ask them to go download files from the state one well at a time as well. To help with this lets use python to simulate a user interacting with a browser in a process know as scraping.

The two styles of scraping that we'll touch on today: with and without a browser.  A third style uses a [web spider](https://scrapy.org/) but we won't get to that today.

With scraping:
-  Check terms of service from the website.
-  Don't scrape agressively as you can cause enough traffic to affect other users. Be a Good Citizen! Don't be a dick. (ie Be Nice)
-  Just plan on the website changing from time to time and having to re-write scrapers.

So let's all take an oath...

---

## Scraping Without a Browser
This is generally a much faster way of collecting data but it doesn't handle data sources that have used features to make it harder to scrape.  In this exercise will be using `geopandas` to get basic information, `requests` to fetch our data, parse that data, then we'll store it to a `.csv` with `pandas`.  We'll walk through how to parse text and **build** a scraper for public data for this example.  After we test it we'll roll it into its automated form with a function.

In [ ]:
# To install packages in the Colab instance that are not normally avalible run a
# command line command with "!"
!pip install geopandas

In [ ]:
import pandas as pd
import requests
from numpy import nan
import geopandas as gpd
from shapely.geometry import Point
import time
import matplotlib.pyplot as plt
import re

%matplotlib inline
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

-  Open the `wells.shp` to a dataframe.
-  Open COGCC's data portal in another tab in our browser. https://cogcc.state.co.us/data.html#/cogis
-  Then navigate to "facility".

Let's load in a dataframe of our Colorado wells and preview the data.

In [ ]:
#Originally from COGCC well spot shapefile - Jackson County
rawurl = 'https://raw.githubusercontent.com/Rocks-n-Code/PythonCourse/master/data/Jackson_057.csv'
apis = pd.read_csv(rawurl)

#Fix raw csv geometry column
def str_to_point(point_string):
  x = int(point_string.split('(')[1].split(' ')[0])
  y = int(point_string.split('(')[1].split(' ')[1].replace(')',''))
  return Point(x,y)

apis['geometry'] = apis['geometry'].apply(str_to_point)

print('Before:',type(apis))

#Change from pandas.DataFrame to geopandas.GeoDataFrame
apis = gpd.GeoDataFrame(apis,
                        geometry='geometry',
                        crs='EPSG:26913')

print('After:',type(apis))
apis.head()

-  Open [COGCC](https://cogcc.state.co.us/data5.html#/cogis_old) in a new tab.
-  On the [website](https://cogcc.state.co.us/data5.html#/cogis_old) select WELL under facility type and select JACKSON county and search.
-  Click on a few wells. Notice that the URL doesn't change.
-  Now this time open a well in a new tab (Right click + 'Open link in new tab').
-  Notice that the URL is now specific to that well.

We're going to utilize this to get more information in a usable format for these wells.  Let's break out the non-unique portions of this URL to use.

In [ ]:
baseURL = 'https://cogcc.state.co.us/cogis/FacilityDetail.asp?facid='
tailURL = '&type=WELL'

Generally websites like this will have a base URL seperated by `?` followed by variables. Notice that COGCC doesn't use the state code in the API number with no deliminator.

In [ ]:
url = baseURL + '05-057-05128'.replace('-','')[2:] + tailURL
print('URL:', url)
r = requests.get(url)
print('Encoding:', r.encoding)
print('RespCode:',type(r.status_code),r.status_code)

A response code of `200` lets us know that it was a good request. No let's look at the text that COGCC sent us back...

In [ ]:
r.text

With the last update of the COGCC's website we can actually send the html to `pandas` directly and recieve a list of dataframes from the page.

In [ ]:
df_list = pd.read_html(r.text)
print('Number of lists found:', len(df_list))
df_list[0]

Now we can see that most of the page's data is avalible in the last table. We'll parse that data down to what we need and define the column names.

In [ ]:
# Select last df in list
tops = df_list[0]

#Find index of the top of the table
i = tops[tops[0] == 'Formation'].index.values[0]

#Set column names without spaces
tops.columns = [x.strip().replace(' ','_') for x in tops.loc[i,:].tolist()]

#Slice dataframe and reset the index
tops = tops[i + 1:].reset_index(drop=True)

#Preview our tops df
tops.head()

Now we'll remove the unit and format the column content as float. I'll use [regular expression](https://docs.python.org/3/howto/regex.html) to do this. `\D` looks for any non-numeric character

In [ ]:
#For only the Log_Top & Log_Bottom columns
for col in tops.columns[1:3]:

  #Where the column is not null remove the non-numeric characters
  tops[col] = tops[col][tops[col].notnull()].apply(lambda x: re.sub('\D',
                                                                    '',
                                                                    x))
  #df[col] = df[col][where not null].apply(lambda x: re.sub(search for, 
  #                                                         replace with,
  #                                                         original string))

  tops[col] = tops[col].astype(float)
print(tops.dtypes)
tops.head()

Now that we have the tops parsed from the website html and formated we'll roll all of that code up into a function.

In [ ]:
def top_parse(text):
  '''
  Input:
  text; str, html code from COGCC facility detail site

  Output
  tops; df, DataFrame of formation tops
  '''
  #Create list of DataFrames
  df_list = pd.read_html(text)

  #Select last DF
  tops = df_list[0]
  
  #Test for no tops
  if 'Formation' not in tops[0].tolist():
    print('No Tops Found')
    return pd.DataFrame()
  
  #Set column names
  i = tops[tops[0] == 'Formation'].index.values[0]
  tops.columns = [x.strip().replace(' ','_') for x in tops.loc[i,:].tolist()]
  tops = tops[i + 1:].reset_index(drop=True)
  #tops = tops[1:].reset_index(drop=True)

  #Format Top and Bottom column
  cols = ['Formation','Log_Top','Log_Bottom','Cored','DSTs']
  tops = tops[cols]
  for col in cols[1:3]:
      tops[col] = tops[col][tops[col].notnull()].apply(lambda x: re.sub('\D',
                                                                    '',
      try:                                                              x))
        tops[col] = tops[col].astype(float)
      except:
        print(col,'float type conversion error.')
  
  tops = tops[tops.Formation != 'No formation data to display.']
  tops = tops[(tops.Formation.notnull())&(~tops[tops.Formation.notnull()].Formation.str.contains('No additional interval'))]
  
  return tops

In [ ]:
print(url)
top_parse(r.text)

And iterrate through our wells. It is _EXTREMELY_ important to add `try` `except` to handle errors in scraping. Scrapers deal with others people's code and things *will* go wrong. It's also a good idea on long scrapes to periodically saveout your progress as there's nothing worse then getting back to something that ran all weeekend pulling data that you need for a project and to see that it crashed.

In [ ]:
topDF = pd.DataFrame()
i = 0
apiSample = apis.head(10) #We'll only do the first few for this example 
total = apiSample.shape[0]

for index, row in apiSample.iterrows(): 
    i += 1
    prec = str(int(100*i/total)) + '% complete  '
    print(row['API_Label'], prec, end='\r')
    try:
        url = baseURL + row['API_Label'].replace('-','')[2:] + tailURL
        print(url)
        r = requests.get(url)

        if r.status_code == 200:
            formations = top_parse(r.text)
            formations['API'] = row['API_Label']
            # topDF = topDF.append(formations,ignore_index=True)
            topDF = pd.concat([topDF, formations],
                               ignore_index=True)
            time.sleep(5) #Wait 5 sec.
        else:
            print(row['API_Label'],':',r.status_code)
    except Exception as e:
        print('Error:',row['API_Label'],e)

topDF.head()

I've gone ahead and pulled all the tops for Jackson County for you.  This took approximately an hour and a half for 771 records to give you an idea of the time needed. These are avalible in the project folder.  This was a basic example with `requests` but if this is something you would like to do regularly I suggest you also check out `urllib`.  There are packages avalible to make the searching and parsing of the html much easier but when you're troubleshooting a tough website it's good to know what you are looking for

---

# Scraping with a Browser with Selenium

Scraping with a browser allows you to navigate around obsticles that are often put in place to discourage scraping, fillout forms, and interact with a website in ways that `requests` can't.  That being said it can be significantly more challenging and can sometimes take much longer. In this example we will pull production data from COGCC. `selenium` locates "elements" of a web page to interact with them to preform tasks. There are several [different methods](https://selenium-python.readthedocs.io/locating-elements.html) to locate elements. We will also use `bs4` to parse a table from html. BeautifulSoup uses tag names and daughter relationships to make finding data easier.  

I've previously written up this function but please open COGCC's [facility search](https://cogcc.state.co.us/cogis/FacilitySearch.asp) in a new tab. Select "Well", enter Weld County's code "123", and the sequence code "39340". Hit search. Select the well that comes up. Note the URL.

With that open, copy the link from the well name.  Notice that there is one of these per wellbore. Paste this url into a new tab. Now let's walk through finding elements & using tags to find the data you need.

In [ ]:
# To selenium Run in Colab
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver

# Set Options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Make Web Driver
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

## If you want to run the code below in a Jupyter Notebook use this create the 
## driver
#from selenium import webdriver

## Make Driver
#chromedriver = "chromedriver.exe" #Path to your chromedriver - https://sites.google.com/a/chromium.org/chromedriver/
#driver = webdriver.Chrome(executable_path=chromedriver)

In [ ]:
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium.webdriver.common.by import By

pd.options.display.max_columns = 50

Elements can be found with `driver.find_elements(By.<method>,<value>)` or individually with `driver.find_element(By.<method>,<value>)`

In [ ]:
print('"By" methods:',dir(By)[:8])

In [ ]:
def pull_CO_prod(api_05, df, driver, pull_excel=False):
    url = 'https://cogcc.state.co.us/cogis/FacilityDetail.asp?facid='+api_05+'&type=WELL'
    print(url)
    driver.get(url)
    time.sleep(1)
    links = driver.find_elements(By.TAG_NAME,'a')
    prod_wellbores = [x.get_attribute("href") for x in driver.find_elements(By.TAG_NAME,'a') if 'production' in x.get_attribute("href")]
    print('prod_wellbores',prod_wellbores)
    for wellbore in prod_wellbores:
        driver.get(wellbore)
        time.sleep(1)
        
        #Download the file
        if pull_excel:
            dwnExcel = driver.find_element(By.XPATH,'//*[@id="mainContent_btnExport"]')
            #//*[@id="mainContent_btnExport"]
            dwnExcel.click()
            
        #Table HTML
        table = driver.find_elements(By.TAG_NAME,'table')[-1]

        #BeautifulSoup
        soup = BeautifulSoup(table.get_attribute('innerHTML'), "html.parser")
        
        rows = soup.find_all('tr')
        row_list = []
        
        #Pull Header 
        for tr in rows[:1]:
            th = tr.find_all('th')
            row = [i.text for i in th]
            row_list.append(row)

        #Pull Rows
        for tr in rows[1:]:
            td = tr.find_all('td')
            row = [i.text.replace('\xa0','') for i in td]
            row_list.append(row)
        
        temp = pd.DataFrame(row_list[1:],columns=row_list[0])
        temp['First of Month'] = pd.to_datetime(temp['First of Month'])
        temp.sort_values(by='First of Month',inplace=True)

        df = pd.concat([df,temp],ignore_index=True)

        return df, driver

# Give it a try

Now that we have the function complete the `for` loop below to feed the individual apis, minus the state code, to the function. Remember that you need to pass the dataframe and the driver to the function too.

Run it for the following wells: `0512339340`,`0512339383`,`0512339370`, & `0512339384`.

In [ ]:
##I've laid out the format for you below. Make edits at *1, *2, & *3.

apis =   #*1: Make a list of your UWI codes

df =  #*2: Make an Empty DataFrame

for api in apis:
    
    api_05 = api[2:]
    print(api_05)
    df, driver =    #*3: Insert the function w/ inputs

df.head()

Once that works for you let's format some of the strings in that dataframe to floats.

In [ ]:
#Set data types & preview data
cols = ['Oil Produced','Gas Produced','Water Volume','Days Produced']
for col in cols:
  df[col].replace('',0,inplace=True)
  df[col] = df[col].astype(float)

df.head()

Plot cumulative oil curves.

In [ ]:
fig=plt.figure(figsize=(15, 5))
ax=fig.add_subplot(111)

for api_wb, group in df[['First of Month','API Sequence','Days Produced','Oil Produced']].groupby('API Sequence'):
  group['CumOil'] = group['Oil Produced'].fillna(0).cumsum()
  
  group['Days Produced'] = group['Days Produced'].replace('',0).astype(float)
  
  group['Total_Days'] = group['Days Produced'].cumsum()
  
  prod_start = df['First of Month'].min()
  group['Elapsed_Days'] = group['First of Month'].apply(lambda x: (x - prod_start).days )
  group['Elapsed_Days'] = group['Elapsed_Days'].astype(float)
  ax.plot(group.Total_Days,
          group.CumOil,
          ls='-',
          label='05-123-'+api_wb,
          fillstyle='none')

plt.legend(loc=2)
plt.show()

---

# COGCCpy

Want all of that data in an easier to use package? Check out [COGCCpy](https://pypi.org/project/COGCCpy/)